# Chat

`Chat` is an object for conversational LLM interactions that tracks history and token usage across single or multiple models.

In [1]:
from irouter import Chat
from irouter.base import nb_markdown

# Load OPENROUTER_API_KEY from .env file
from dotenv import load_dotenv

load_dotenv()

True

In this notebook we will use free tiers for Moonshot AI's Kimi K2 and Google's Gemma 3N. 

An overview of all available models can be found by calling `get_all_models`:
```python
from irouter.base import get_all_models
model_slugs = get_all_models()
model_slugs
```

In [2]:
model_names = ["moonshotai/kimi-k2:free", "google/gemma-3n-e2b-it:free"]

# Test conversation messages
first_message = "Who played the guitar solo on Steely Dan's Kid Charlemagne?"
second_message = "What other songs did this guitarist play on with Steely Dan?"

# Single Model

The simplest way to use `Chat` is with a single LLM by providing a model slug. Unlike `Call`, `Chat` maintains conversation history and tracks token usage.

In this example we initialize a `Chat` object with the free tier of Moonshot AI's Kimi-K2 LLM.

To set the API key you can either set an environment variable for `OPENROUTER_API_KEY` to your project or pass `api_key` when initializing `Chat`.

In [3]:
chat = Chat(model_names[0])
print(f"System prompt: {chat.system}")
print(f"Initial history length: {len(chat.history(model_names[0]))}")

System prompt: You are a helpful assistant.
Initial history length: 1


Let's start a conversation and see how history is tracked:

In [4]:
response1 = chat(first_message)
nb_markdown(response1)

TypeError: Call._get_resp() missing 1 required positional argument: 'raw'

Now let's check the conversation history and token usage:

In [ ]:
print(f"History length after first message: {len(chat.history(model_names[0]))}")
print(f"\nConversation history:")
for i, msg in enumerate(chat.history(model_names[0])):
    print(f"{i + 1}. [{msg['role']}]: {msg['content'][:50]}...")

print(f"\nToken usage: {chat.usage[model_names[0]]}")

Let's continue the conversation to see how context is maintained:

In [ ]:
response2 = chat(second_message)
nb_markdown(response2)

In [ ]:
print(f"History length after second message: {len(chat.history(model_names[0]))}")
print(f"\nUpdated conversation history:")
for i, msg in enumerate(chat.history(model_names[0])):
    role_color = (
        "\033[94m"
        if msg["role"] == "user"
        else "\033[92m"
        if msg["role"] == "assistant"
        else "\033[93m"
    )
    reset_color = "\033[0m"
    print(
        f"{i + 1}. {role_color}[{msg['role']}]{reset_color}: {msg['content'][:80]}..."
    )

print(f"\nCumulative token usage: {chat.usage[model_names[0]]}")

## Multiple Models

Using multiple models with `Chat` allows you to compare responses while maintaining separate conversation histories and tracking usage per model.

In [ ]:
multi_chat = Chat(model_names, system="You are a music expert assistant.")
print(f"Models: {multi_chat.model}")
print(f"System prompt: {multi_chat.system}")
print(
    f"Initial histories: {[len(multi_chat.history(model)) for model in multi_chat.model]}"
)

When multiple models are used, `Chat` returns a list of responses and maintains separate histories for each model:

In [ ]:
multi_responses1 = multi_chat(first_message)
print(f"Number of responses: {len(multi_responses1)}")
print(f"\nResponses:")
for i, (model, response) in enumerate(zip(multi_chat.model, multi_responses1)):
    print(f"\n{i + 1}. {model}:")
    print(f"   {response[:100]}...")

Let's examine the separate histories and usage for each model:

In [ ]:
for model in multi_chat.model:
    print(f"\n=== {model} ===")
    print(f"History length: {len(multi_chat.history(model))}")
    print(f"Token usage: {multi_chat.usage[model]}")
    print(f"Last assistant message: {multi_chat.history(model)[-1]['content'][:80]}...")

Let's continue the conversation with both models to see how context is maintained separately:

In [ ]:
multi_responses2 = multi_chat(second_message)
print("Follow-up responses:")
for i, (model, response) in enumerate(zip(multi_chat.model, multi_responses2)):
    print(f"\n{i + 1}. {model}:")
    nb_markdown(response)
    print("---")

## Usage Tracking Summary

One of the key advantages of `Chat` over `Call` is comprehensive usage tracking per model:

In [ ]:
print("=== USAGE SUMMARY ===")
total_tokens = 0
for model in multi_chat.model:
    usage = multi_chat.usage[model]
    print(f"\n{model}:")
    print(f"  Conversation turns: {(len(multi_chat.history[model]) - 1) // 2}")
    print(f"  Prompt tokens: {usage['prompt_tokens']}")
    print(f"  Completion tokens: {usage['completion_tokens']}")
    print(f"  Total tokens: {usage['total_tokens']}")
    total_tokens += usage["total_tokens"]

print(f"\nGrand total tokens across all models: {total_tokens}")

## Accessing Individual Model Responses

You can easily access responses from specific models:

In [ ]:
# Get response from specific model by index
kimi_response = multi_responses2[0]
gemma_response = multi_responses2[1]

print("Kimi K2 response:")
nb_markdown(kimi_response)
print("\n" + "=" * 50 + "\n")
print("Gemma 3N response:")
nb_markdown(gemma_response)

## Model Comparison

The separate history tracking allows for easy comparison of how different models handle the same conversation:

In [ ]:
print("=== MODEL COMPARISON ===")
for i, model in enumerate(multi_chat.model):
    history = multi_chat.history(model)
    usage = multi_chat.usage[model]

    print(f"\n{i + 1}. {model}:")
    print(f"   Messages in history: {len(history)}")
    print(
        f"   Average tokens per response: {usage['completion_tokens'] / ((len(history) - 1) // 2):.1f}"
    )
    print(
        f"   Efficiency (completion/prompt ratio): {usage['completion_tokens'] / usage['prompt_tokens']:.3f}"
    )

## Summary

`Chat` provides powerful conversational AI capabilities with:

- **History tracking**: Maintains separate conversation histories for each model
- **Usage tracking**: Comprehensive token usage statistics per model  
- **Multi-model support**: Compare responses from multiple models simultaneously
- **Context awareness**: Follow-up questions use full conversation context

This makes `Chat` ideal for applications requiring conversational context, usage monitoring, or model comparison workflows.